<a href="https://colab.research.google.com/github/FengruiJing/HIV_Prediction/blob/main/HIV_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Testing on Google Colab
https://www.youtube.com/watch?v=ULFlFznyUrc

# Mount google drive

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [ ]:
#list the files
!ls

HIV_RNN.ipynb
MovingWindowsTotalDataPredictionHIVDisRandom100d.csv
MovingWindowsTotalDataPredictionHIVDisRandom30d.csv
MovingWindowsTotalDataPredictionHIVDisRandom50d.csv
MovingWindowsTotalDataPredictionHIVPCIDisRandom30d.csv
MovingWindowsTotalDataPredictionHIVPCIDisRandom50d.csv
MovingWindowsTotalDataPredictionHIVPCIRandom100b.csv
MovingWindowsTotalDataPredictionHIVPCIRandom30b.csv
MovingWindowsTotalDataPredictionHIVPCIRandom50b.csv
MovingWindowsTotalDataPredictionHIVPCISCIDisRandom30d.csv
MovingWindowsTotalDataPredictionHIVPCISCIDisRandom50d.csv
MovingWindowsTotalDataPredictionHIVPCISCIRandom30d.csv
MovingWindowsTotalDataPredictionHIVPCISCIRandom50d.csv
MovingWindowsTotalDataPredictionHIVRandom100a.csv
MovingWindowsTotalDataPredictionHIVRandom30a.csv
MovingWindowsTotalDataPredictionHIVRandom50a.csv
MovingWindowsTotalDataPredictionHIVSCIDisRandom30d.csv
MovingWindowsTotalDataPredictionHIVSCIDisRandom50d.csv
MovingWindowsTotalDataPredictionHIVSCIRandom100c.csv
MovingWindowsTotalDataPre

In [ ]:
!pip install howdoi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.5/312.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 35.9 MB/s eta 0:00:00
  Created wheel for howdoi: filename=howdoi-2.0.20-py3-none-any.whl size=16076 sha256=d43cb23f106c3dc17969eaaf1702357e57e0a3724c43746e37c4a177b3f2128f
  Stored in directory: /root/.cache/pip/wheels/31/a1/c8/80acaaa566a958b7dc3da1516e08ffd8caa45cfdd229b6ab8c
Successfully built howdoi
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handlin

In [ ]:

import pandas as pd

df = pd.read_csv("TotalDataTop20.csv")
print(df)
df.to_csv("TotalDataSaveExample.csv",index=False)

     County  Year  HIVprevalence     HIVXPCI1       HIVSCI1     HIVDis1
0     12125  2008         3395.5  3821045.108  1.630000e+11  357.155598
1     12125  2009         3521.6  3956743.323  1.690000e+11  368.891022
2     12125  2010         2435.8  2849447.944  1.200000e+11  320.329009
3     12125  2011         2425.2  2880486.549  1.210000e+11  373.287738
4     12125  2012         4887.6  5302853.009  2.280000e+11  359.410945
..      ...   ...            ...          ...           ...         ...
235   29510  2015         1081.1  4391209.991  5.020870e+09  215.117847
236   29510  2016         1092.7  4468866.467  5.091784e+09  221.102532
237   29510  2017         1105.3  4578251.667  5.207725e+09  228.967835
238   29510  2018         1122.2  4671352.207  5.332183e+09  238.750427
239   29510  2019         1058.1        0.000  0.000000e+00    0.000000

[240 rows x 6 columns]


In [ ]:
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataLast20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 101, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 42123
1/1 [==============================] - 0s 22ms/step
For 10 neurons: MAE: 9.474531718095145, MAPE: 0.4888622544751864
1/1 [==============================] - 0s 33ms/step
For 20 neurons: MAE: 34.93139917055766, MAPE: 1.7031834215906905
1/1 [==============================] - 0s 22ms/step
For 30 neurons: MAE: 60.92351915041605, MAPE: 2.9424259053567954
1/1 [==============================] - 0s 19ms/step
For 40 neurons: MAE: 61.7598823706309, MAPE: 2.9847778052306793
1/1 [==============================] - 0s 20ms/step
For 50 neurons: MAE: 115.03762734731038, MAPE: 5.5258554569950755
1/1 [==============================] - 0s 20ms/step
For 60 neurons: MAE: 95.88543510437012, MAPE: 4.6108504926041745
1/1 [==============================] - 0s 19ms/step
For 70 neurons: MAE: 71.76468601226806, MAPE: 3.4579142973677683
1/1 [==============================] - 0s 28ms/step
For 80 neurons: MAE: 81.35239677429199, MAPE: 3.9161927166982227
1/1 [==============================] - 0

In [ ]:

#####HIV
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)
df = pd.read_csv("TotalDataLast20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 101, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0]

            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year]['HIVprevalence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsWindows9TotalTop20_HIV.csv', index=False)

Processing county 42123
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 4.011380132039388, MAPE: 0.20463350397672012
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 3.4674100240071617, MAPE: 0.16238497832964904
1/1 [==============================] - 0s 30ms/step
For 30 neurons: MAE: 3.3968718210856124, MAPE: 0.15867097585925452
1/1 [==============================] - 0s 22ms/step
For 40 neurons: MAE: 3.5975861867268875, MAPE: 0.14478933728951257
1/1 [==============================] - 0s 28ms/step
For 50 neurons: MAE: 27.70101038614909, MAPE: 1.2964004716569482
1/1 [==============================] - 0s 22ms/step
For 60 neurons: MAE: 31.772487958272297, MAPE: 1.4905331409718119
1/1 [==============================] - 0s 21ms/step
For 70 neurons: MAE: 55.046304066975914, MAPE: 2.602938299696651
1/1 [==============================] - 0s 25ms/step
For 80 neurons: MAE: 25.902695814768474, MAPE: 1.2097473318236318
1/1 [==========================

In [ ]:
#####HIV
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)
df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 101, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0]

            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year]['HIVprevalence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsWindows9TotalTop20_HIV.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 20ms/step
For 10 neurons: MAE: 205.26811523437496, MAPE: 0.09624902340888297
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 697.9294921874999, MAPE: 0.3264303744920553
1/1 [==============================] - 0s 19ms/step
For 30 neurons: MAE: 779.1767903645833, MAPE: 0.36439114500254455
1/1 [==============================] - 0s 20ms/step
For 40 neurons: MAE: 424.61650390625, MAPE: 0.19859805091095414


1/1 [==============================] - 0s 24ms/step
For 50 neurons: MAE: 329.9630045572917, MAPE: 0.15424931831678765


1/1 [==============================] - 0s 20ms/step
For 60 neurons: MAE: 329.4216796875, MAPE: 0.1540350917984438
1/1 [==============================] - 0s 23ms/step
For 70 neurons: MAE: 1061.2060221354166, MAPE: 0.4961631090483458
1/1 [==============================] - 0s 28ms/step
For 80 neurons: MAE: 359.9446940104167, MAPE: 0.16826143251143352
1/1 [==============================] - 0s 20ms/step
For 90 neurons: MAE: 287.579638671875, MAPE: 0.13447286443493575
1/1 [==============================] - 0s 18ms/step
For 100 neurons: MAE: 385.9444091796875, MAPE: 0.18041697338727536
Best model for county 12125 had 10 neurons with MAE: 205.26811523437496, MAPE: 0.09624902340888297
Processing county 11001
1/1 [==============================] - 0s 29ms/step
For 10 neurons: MAE: 36.04827473958343, MAPE: 0.015462645836578479
1/1 [==============================] - 0s 21ms/step
For 20 neurons: MAE: 220.5159993489583, MAPE: 0.09469118168300435
1/1 [==============================] - 0s 32ms/step
Fo

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.8581515009998384
GPU (s):
0.09358194200012804
GPU speedup over CPU: 41x


In [ ]:
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 163.79196597728878, MAPE: 0.07678931176133082
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 507.8207878291608, MAPE: 0.23753288172546064
1/1 [==============================] - 0s 20ms/step
For 30 neurons: MAE: 813.4755629877249, MAPE: 0.38038265045012004
1/1 [==============================] - 0s 21ms/step
For 40 neurons: MAE: 261.19701592822884, MAPE: 0.1221183006798499
1/1 [==============================] - 0s 31ms/step
For 50 neurons: MAE: 1417.277130933603, MAPE: 0.6626079392911588
Best model for county 12125 had 10 neurons with MAE: 163.79196597728878, MAPE: 0.07678931176133082
Processing county 11001
1/1 [==============================] - 0s 27ms/step
For 10 neurons: MAE: 35.46896869701656, MAPE: 0.015068582040157576
1/1 [==============================] - 0s 22ms/step
For 20 neurons: MAE: 328.9160525083541, MAPE: 0.1409161183154265
1/1 [======================

In [ ]:
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=2, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 96.84419655412603, MAPE: 0.045403197745078026
1/1 [==============================] - 0s 21ms/step
For 20 neurons: MAE: 201.0423996279636, MAPE: 0.094034717516705
1/1 [==============================] - 0s 28ms/step
For 30 neurons: MAE: 122.29848016997191, MAPE: 0.057166867037650365
1/1 [==============================] - 0s 22ms/step
For 40 neurons: MAE: 485.5027046531438, MAPE: 0.22703420617787984
1/1 [==============================] - 0s 21ms/step
For 50 neurons: MAE: 838.363646191359, MAPE: 0.39195897641574723
Best model for county 12125 had 10 neurons with MAE: 96.84419655412603, MAPE: 0.045403197745078026
Processing county 11001
1/1 [==============================] - 0s 26ms/step
For 10 neurons: MAE: 104.74312020366399, MAPE: 0.04497163218395194
1/1 [==============================] - 0s 38ms/step
For 20 neurons: MAE: 156.10775800347332, MAPE: 0.06681165358523095
1/1 [====================

In [ ]:
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 22ms/step
For 10 neurons: MAE: 189.77502958985679, MAPE: 0.08894412249901139
1/1 [==============================] - 0s 38ms/step
For 20 neurons: MAE: 82.44457727310571, MAPE: 0.03851847352292214
1/1 [==============================] - 0s 24ms/step
For 30 neurons: MAE: 406.45972281992425, MAPE: 0.19013622098789815
1/1 [==============================] - 0s 32ms/step
For 40 neurons: MAE: 1389.5218809326495, MAPE: 0.6497185313790813
1/1 [==============================] - 0s 29ms/step
For 50 neurons: MAE: 659.6192034900192, MAPE: 0.30837828025947095
Best model for county 12125 had 20 neurons with MAE: 82.44457727310571, MAPE: 0.03851847352292214
Processing county 11001
1/1 [==============================] - 0s 20ms/step
For 10 neurons: MAE: 130.30929958870033, MAPE: 0.056041998498623374
1/1 [==============================] - 0s 27ms/step
For 20 neurons: MAE: 554.5907273570696, MAPE: 0.23774349189994026
1/1 [===================

In [ ]:
#startfrom2016
###HIVPCI
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalDataTop20.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-8:i] for i in range(8, len(train))])
    y_train = np.array([train[i, 0] for i in range(8, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2016, 2020):
            input_data = train[-8:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 8, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2016) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
#predictions_df.to_csv('D:/Data/Paper_HIV/FinalExperimentHIVPrevalencePrediction/predictionsHIVprevalence2019_onlyHIVprevalenceHIVPCISCIBest.csv', index=False)

Processing county 12125
1/1 [==============================] - 0s 36ms/step
For 10 neurons: MAE: 505.45090456143976, MAPE: 0.235363690793981
1/1 [==============================] - 0s 32ms/step
For 20 neurons: MAE: 348.084899256006, MAPE: 0.15945652977602398
1/1 [==============================] - 0s 23ms/step
For 30 neurons: MAE: 514.7906872406601, MAPE: 0.2378993746547166
1/1 [==============================] - 0s 28ms/step
For 40 neurons: MAE: 2099.8708999753003, MAPE: 0.9673382606037841
1/1 [==============================] - 0s 23ms/step
For 50 neurons: MAE: 1317.3455187976365, MAPE: 0.6071469690856702
Best model for county 12125 had 20 neurons with MAE: 348.084899256006, MAPE: 0.15945652977602398
Processing county 11001
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 243.4809454154223, MAPE: 0.10392215402719238
1/1 [==============================] - 0s 31ms/step
For 20 neurons: MAE: 402.55298034846777, MAPE: 0.1710682304182624
1/1 [===========================

## MovingWindowsTotalDataPredictionHIVPCI

In [ ]:
###HIVPCI total data
# all counties with best results.
#####################
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCI.csv", index=False)

Processing county 1001


KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

for county in df['County'].unique():
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCI.csv", index=False)

Processing county 1001
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 12.659154701232902, MAPE: 0.05624458357789958
1/1 [==============================] - 0s 22ms/step
For 20 neurons: MAE: 35.94431738853451, MAPE: 0.1586563938414116
1/1 [==============================] - 0s 21ms/step
For 30 neurons: MAE: 20.86838315725326, MAPE: 0.09199654287574266
1/1 [==============================] - 0s 23ms/step
For 40 neurons: MAE: 1.2048907995224074, MAPE: 0.005284453431479457


1/1 [==============================] - 0s 21ms/step
For 50 neurons: MAE: 11.738078045845024, MAPE: 0.05183074252633233
Best model for county 1001 had 40 neurons with MAE: 1.2048907995224074, MAPE: 0.005284453431479457
Processing county 1003


1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 1.7224415699640758, MAPE: 0.010124969498527562
1/1 [==============================] - 0s 25ms/step
For 20 neurons: MAE: 4.295485532283796, MAPE: 0.02500787172354907
1/1 [==============================] - 0s 23ms/step
For 30 neurons: MAE: 7.647591857115401, MAPE: 0.04510714834044807
1/1 [==============================] - 0s 30ms/step
For 40 neurons: MAE: 25.671763268113153, MAPE: 0.1504601373813391
1/1 [==============================] - 0s 21ms/step
For 50 neurons: MAE: 4.309999418258656, MAPE: 0.025436964532678016
Best model for county 1003 had 10 neurons with MAE: 1.7224415699640758, MAPE: 0.010124969498527562
Processing county 1005
1/1 [==============================] - 0s 38ms/step
For 10 neurons: MAE: 39.16879442135496, MAPE: 0.08605374130060546
1/1 [==============================] - 0s 24ms/step
For 20 neurons: MAE: 177.34509797493615, MAPE: 0.38296296609124675
1/1 [==============================] - 0s 29ms/s

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# **Randomly select 50 counties**

In [ ]:
#HIV+HIVPCI
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(0)

# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 50, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCIRandom50b.csv", index=False)


Processing county 6097
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 11.183374428749062, MAPE: 0.03426507488517345
1/1 [==============================] - 0s 35ms/step
For 20 neurons: MAE: 39.774995946884154, MAPE: 0.12121449180624061
1/1 [==============================] - 0s 20ms/step
For 30 neurons: MAE: 99.47933540344239, MAPE: 0.30266379602672583
1/1 [==============================] - 0s 30ms/step
For 40 neurons: MAE: 103.72665791511537, MAPE: 0.3157812279956134
1/1 [==============================] - 0s 27ms/step
For 50 neurons: MAE: 56.505274057388306, MAPE: 0.17237443172170097
Best model for county 6097 had 10 neurons with MAE: 11.183374428749062, MAPE: 0.03426507488517345
Processing county 39101
1/1 [==============================] - 0s 30ms/step
For 10 neurons: MAE: 26.779645520448685, MAPE: 0.1353022451839022
1/1 [==============================] - 0s 20ms/step
For 20 neurons: MAE: 13.88433446288109, MAPE: 0.07338685461676807
1/1 [=====================

In [ ]:
#####HIV
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)
df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(0)
# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 50, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0]

            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year]['HIVprevalence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVRandom50a.csv", index=False)


Processing county 6097
1/1 [==============================] - 0s 22ms/step
For 10 neurons: MAE: 22.487465413411446, MAPE: 0.06820010933866832
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 87.34193725585936, MAPE: 0.26541046818403996
1/1 [==============================] - 0s 23ms/step
For 30 neurons: MAE: 190.0733723958333, MAPE: 0.5781172549272748
1/1 [==============================] - 0s 23ms/step
For 40 neurons: MAE: 100.53275349934894, MAPE: 0.30613767576471024


1/1 [==============================] - 0s 20ms/step
For 50 neurons: MAE: 288.46454671223955, MAPE: 0.8778944021582197
Best model for county 6097 had 10 neurons with MAE: 22.487465413411446, MAPE: 0.06820010933866832
Processing county 39101


1/1 [==============================] - 0s 24ms/step
For 10 neurons: MAE: 20.17445780436198, MAPE: 0.10271597410122657
1/1 [==============================] - 0s 20ms/step
For 20 neurons: MAE: 17.31163228352864, MAPE: 0.09395267619400587
1/1 [==============================] - 0s 21ms/step
For 30 neurons: MAE: 35.16356608072916, MAPE: 0.18649379124313661
1/1 [==============================] - 0s 19ms/step
For 40 neurons: MAE: 40.30553894042968, MAPE: 0.21428468970098727
1/1 [==============================] - 0s 20ms/step
For 50 neurons: MAE: 127.90000610351562, MAPE: 0.6714311475068321
Best model for county 39101 had 20 neurons with MAE: 17.31163228352864, MAPE: 0.09395267619400587
Processing county 12073
1/1 [==============================] - 0s 29ms/step
For 10 neurons: MAE: 13.31329345703125, MAPE: 0.02699142923182146
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 33.88322753906251, MAPE: 0.06907375348380425
1/1 [==============================] - 0s 21ms/step


In [ ]:
#HIV+HIVSCI
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(0)

# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 50, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVSCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVSCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVSCIRandom50c.csv", index=False)

Processing county 6097
1/1 [==============================] - 0s 32ms/step
For 10 neurons: MAE: 13.157802764574702, MAPE: 0.04019704788072798
1/1 [==============================] - 0s 20ms/step
For 20 neurons: MAE: 36.96427156925201, MAPE: 0.11258170272749167
1/1 [==============================] - 0s 27ms/step
For 30 neurons: MAE: 77.4103818655014, MAPE: 0.23565293957837063
1/1 [==============================] - 0s 26ms/step
For 40 neurons: MAE: 117.67501187324525, MAPE: 0.35828039732617895


1/1 [==============================] - 0s 29ms/step
For 50 neurons: MAE: 64.14512181282043, MAPE: 0.1954466234300062
Best model for county 6097 had 10 neurons with MAE: 13.157802764574702, MAPE: 0.04019704788072798
Processing county 39101


1/1 [==============================] - 0s 26ms/step
For 10 neurons: MAE: 13.3487197637558, MAPE: 0.0695987452971887
1/1 [==============================] - 0s 20ms/step
For 20 neurons: MAE: 26.95186458826065, MAPE: 0.13384247152689832
1/1 [==============================] - 0s 23ms/step
For 30 neurons: MAE: 59.663924632469815, MAPE: 0.3024349260897538
1/1 [==============================] - 0s 20ms/step
For 40 neurons: MAE: 43.351087761918706, MAPE: 0.21806931836821217
1/1 [==============================] - 0s 24ms/step
For 50 neurons: MAE: 90.40244230826693, MAPE: 0.47641855497446745
Best model for county 39101 had 10 neurons with MAE: 13.3487197637558, MAPE: 0.0695987452971887
Processing county 12073
1/1 [==============================] - 0s 19ms/step
For 10 neurons: MAE: 15.352144370476404, MAPE: 0.030899666312500573
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 44.287079528967546, MAPE: 0.08862909673629016
1/1 [==============================] - 0s 23ms/step


In [ ]:
#HIV+HIVDis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(0)

# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 50, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVDisRandom50d.csv", index=False)

Processing county 6097
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 30.82619199752806, MAPE: 0.0933204037715064
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 26.070963001251204, MAPE: 0.07954210069375002
1/1 [==============================] - 0s 36ms/step
For 30 neurons: MAE: 49.60696897506714, MAPE: 0.15111415073255
1/1 [==============================] - 0s 24ms/step
For 40 neurons: MAE: 40.62166185379027, MAPE: 0.12411610544466485
1/1 [==============================] - 0s 33ms/step
For 50 neurons: MAE: 153.7583661079407, MAPE: 0.46808119377771346
Best model for county 6097 had 20 neurons with MAE: 26.070963001251204, MAPE: 0.07954210069375002
Processing county 39101
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 13.858743298053744, MAPE: 0.07309198452889483
1/1 [==============================] - 0s 28ms/step
For 20 neurons: MAE: 24.28119883139928, MAPE: 0.13225645943136485
1/1 [==========================

In [ ]:
#HIV+PCI+SCI
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(0)

# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 50, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVSCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVSCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCISCIRandom50d.csv", index=False)

Processing county 6097
1/1 [==============================] - 0s 28ms/step
For 10 neurons: MAE: 24.84026477336882, MAPE: 0.07533904796347277
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 18.93408500353496, MAPE: 0.05702211257284343
1/1 [==============================] - 0s 27ms/step
For 30 neurons: MAE: 40.173387241363514, MAPE: 0.12257270268356141
1/1 [==============================] - 0s 20ms/step
For 40 neurons: MAE: 31.472703289985645, MAPE: 0.09618671885550019


1/1 [==============================] - 0s 30ms/step
For 50 neurons: MAE: 62.94850380420686, MAPE: 0.19205372315390634
Best model for county 6097 had 20 neurons with MAE: 18.93408500353496, MAPE: 0.05702211257284343
Processing county 39101


1/1 [==============================] - 0s 25ms/step
For 10 neurons: MAE: 11.570516246557238, MAPE: 0.061370125835209434
1/1 [==============================] - 0s 29ms/step
For 20 neurons: MAE: 26.856193590164185, MAPE: 0.1320673523333035
1/1 [==============================] - 0s 21ms/step
For 30 neurons: MAE: 13.115506696701052, MAPE: 0.07134413685299197
1/1 [==============================] - 0s 28ms/step
For 40 neurons: MAE: 36.129635361830395, MAPE: 0.18148155227792703
1/1 [==============================] - 0s 19ms/step
For 50 neurons: MAE: 15.991047072410586, MAPE: 0.08401422352320272
Best model for county 39101 had 10 neurons with MAE: 11.570516246557238, MAPE: 0.061370125835209434
Processing county 12073
1/1 [==============================] - 0s 20ms/step
For 10 neurons: MAE: 14.243984409173303, MAPE: 0.029053088446330047
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 66.71198253134888, MAPE: 0.1340157164744061
1/1 [==============================] - 0s 22

In [ ]:
#HIV+PCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(0)

# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 50, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCIDisRandom50d.csv", index=False)

Processing county 6097
1/1 [==============================] - 0s 18ms/step
For 10 neurons: MAE: 15.628212976455663, MAPE: 0.047671783702864436
1/1 [==============================] - 0s 32ms/step
For 20 neurons: MAE: 42.72886462211608, MAPE: 0.1300968077293354
1/1 [==============================] - 0s 22ms/step
For 30 neurons: MAE: 38.514893174171426, MAPE: 0.11742507367872453
1/1 [==============================] - 0s 28ms/step
For 40 neurons: MAE: 89.28880510330202, MAPE: 0.27202471946065704
1/1 [==============================] - 0s 23ms/step
For 50 neurons: MAE: 130.8259363651276, MAPE: 0.3984316895231464
Best model for county 6097 had 10 neurons with MAE: 15.628212976455663, MAPE: 0.047671783702864436
Processing county 39101
1/1 [==============================] - 0s 28ms/step
For 10 neurons: MAE: 11.878155845403674, MAPE: 0.06297040647544767
1/1 [==============================] - 0s 28ms/step
For 20 neurons: MAE: 14.879443007707598, MAPE: 0.07927869503673589
1/1 [====================

In [ ]:
#HIV+SCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(0)

# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 50, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVDis1', 'HIVSCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVDis1', 'HIVSCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVSCIDisRandom50d.csv", index=False)

Processing county 6097
1/1 [==============================] - 0s 23ms/step
For 10 neurons: MAE: 8.47391043106713, MAPE: 0.026206407633133293
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 14.838618354002657, MAPE: 0.044663187728740845
1/1 [==============================] - 0s 20ms/step
For 30 neurons: MAE: 23.150581574439986, MAPE: 0.07077347548525441
1/1 [==============================] - 0s 20ms/step
For 40 neurons: MAE: 16.315407490730326, MAPE: 0.04914984883561587


1/1 [==============================] - 0s 18ms/step
For 50 neurons: MAE: 109.4901926994324, MAPE: 0.3337582832944483
Best model for county 6097 had 10 neurons with MAE: 8.47391043106713, MAPE: 0.026206407633133293
Processing county 39101


1/1 [==============================] - 0s 20ms/step
For 10 neurons: MAE: 26.91022463440895, MAPE: 0.13626843812801195
1/1 [==============================] - 0s 24ms/step
For 20 neurons: MAE: 37.09706347237031, MAPE: 0.1825595935096628
1/1 [==============================] - 0s 23ms/step
For 30 neurons: MAE: 13.00922437310219, MAPE: 0.07007022487919934
1/1 [==============================] - 0s 21ms/step
For 40 neurons: MAE: 21.258958446979523, MAPE: 0.10567629079786996
1/1 [==============================] - 0s 19ms/step
For 50 neurons: MAE: 20.38670239051183, MAPE: 0.11350586640963069
Best model for county 39101 had 30 neurons with MAE: 13.00922437310219, MAPE: 0.07007022487919934
Processing county 12073
1/1 [==============================] - 0s 18ms/step
For 10 neurons: MAE: 31.100089208285, MAPE: 0.06228125297667308
1/1 [==============================] - 0s 21ms/step
For 20 neurons: MAE: 31.370039546489693, MAPE: 0.06382432339651498
1/1 [==============================] - 0s 31ms/step
F

In [ ]:
#HIV+PCI+SCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(0)

# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 50, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 4)) # 4 because we now have 4 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 4))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 3), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCISCIDisRandom50d.csv", index=False)

Processing county 6097
1/1 [==============================] - 0s 24ms/step
For 10 neurons: MAE: 9.518557671705878, MAPE: 0.02919200418155737
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 38.487894558906554, MAPE: 0.11723661960742432
1/1 [==============================] - 0s 37ms/step
For 30 neurons: MAE: 13.627035633722926, MAPE: 0.04107466606169883
1/1 [==============================] - 0s 21ms/step
For 40 neurons: MAE: 58.476286458969106, MAPE: 0.17849782674031087
1/1 [==============================] - 0s 41ms/step
For 50 neurons: MAE: 16.718034696579025, MAPE: 0.05042236155256935
Best model for county 6097 had 10 neurons with MAE: 9.518557671705878, MAPE: 0.02919200418155737
Processing county 39101
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 23.060867685079575, MAPE: 0.11733983409398374
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 24.571345421671868, MAPE: 0.12275859562551943
1/1 [===================

# Randomly select 30 counties

In [ ]:
#HIV+HIVPCI
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(1)

# Randomly select 30 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 30, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCIRandom30b.csv", index=False)

Processing county 26035
1/1 [==============================] - 0s 24ms/step
For 10 neurons: MAE: 9.26674669384957, MAPE: 0.07017626513019842
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 37.879584463437396, MAPE: 0.3016726647368429
1/1 [==============================] - 0s 20ms/step
For 30 neurons: MAE: 72.93679152218004, MAPE: 0.5744019140111946
1/1 [==============================] - 0s 19ms/step
For 40 neurons: MAE: 41.74178252319495, MAPE: 0.3269051030879094


1/1 [==============================] - 0s 31ms/step
For 50 neurons: MAE: 37.347974566618596, MAPE: 0.2934193188181811
Best model for county 26035 had 10 neurons with MAE: 9.26674669384957, MAPE: 0.07017626513019842
Processing county 51155


1/1 [==============================] - 0s 87ms/step
For 10 neurons: MAE: 6.489921689033513, MAPE: 0.09331361996667276
1/1 [==============================] - 0s 55ms/step
For 20 neurons: MAE: 5.691771057248118, MAPE: 0.07413085256815942
1/1 [==============================] - 0s 77ms/step
For 30 neurons: MAE: 4.455871323744461, MAPE: 0.0629816616991448
1/1 [==============================] - 0s 60ms/step
For 40 neurons: MAE: 21.584634518623364, MAPE: 0.2847780828049306
1/1 [==============================] - 0s 21ms/step
For 50 neurons: MAE: 28.587705628077206, MAPE: 0.3784080226323792
Best model for county 51155 had 30 neurons with MAE: 4.455871323744461, MAPE: 0.0629816616991448
Processing county 55013
1/1 [==============================] - 0s 44ms/step
For 10 neurons: MAE: 10.902214039117098, MAPE: 0.23682475173369755
1/1 [==============================] - 0s 50ms/step
For 20 neurons: MAE: 20.399455842375755, MAPE: 0.4352040214990303
1/1 [==============================] - 0s 38ms/step
F

In [ ]:
#####HIV
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)
df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(1)
# Randomly select 30 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 30, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0]

            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year]['HIVprevalence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVRandom30a.csv", index=False)

Processing county 26035
1/1 [==============================] - 0s 22ms/step
For 10 neurons: MAE: 5.475521850585939, MAPE: 0.04499234099553907
1/1 [==============================] - 0s 40ms/step
For 20 neurons: MAE: 42.34082946777343, MAPE: 0.33626853236215376
1/1 [==============================] - 0s 28ms/step
For 30 neurons: MAE: 19.750639851888014, MAPE: 0.1592321912910414
1/1 [==============================] - 0s 47ms/step
For 40 neurons: MAE: 71.4468780517578, MAPE: 0.5693788089934341
1/1 [==============================] - 0s 30ms/step
For 50 neurons: MAE: 52.602248128255205, MAPE: 0.41986954952659095
Best model for county 26035 had 10 neurons with MAE: 5.475521850585939, MAPE: 0.04499234099553907
Processing county 51155
1/1 [==============================] - 0s 64ms/step
For 10 neurons: MAE: 4.778721618652345, MAPE: 0.0689685160785634
1/1 [==============================] - 0s 25ms/step
For 20 neurons: MAE: 8.049466451009115, MAPE: 0.11491911348224433
1/1 [=========================

In [ ]:
#HIV+HIVSCI
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(1)

# Randomly select 30 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 30, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVSCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVSCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVSCIRandom30c.csv", index=False)

Processing county 26035
1/1 [==============================] - 0s 28ms/step
For 10 neurons: MAE: 8.891864232222247, MAPE: 0.06850419538919679
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 20.41004879077275, MAPE: 0.15672010751881857
1/1 [==============================] - 0s 22ms/step
For 30 neurons: MAE: 8.289571630954745, MAPE: 0.06210916699004107
1/1 [==============================] - 0s 22ms/step
For 40 neurons: MAE: 24.789613517125446, MAPE: 0.200005715282042
1/1 [==============================] - 0s 30ms/step
For 50 neurons: MAE: 40.62434380054473, MAPE: 0.3252373044802485
Best model for county 26035 had 30 neurons with MAE: 8.289571630954745, MAPE: 0.06210916699004107
Processing county 51155
1/1 [==============================] - 0s 25ms/step
For 10 neurons: MAE: 5.582661437988288, MAPE: 0.08018115888120039
1/1 [==============================] - 0s 32ms/step
For 20 neurons: MAE: 7.0781028777361, MAPE: 0.09075410149452408
1/1 [===========================

In [ ]:
#HIV+HIVDis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(1)

# Randomly select 30 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 30, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVDisRandom30d.csv", index=False)

Processing county 26035
1/1 [==============================] - 0s 23ms/step
For 10 neurons: MAE: 7.320577808221192, MAPE: 0.05939712963718082
1/1 [==============================] - 0s 26ms/step
For 20 neurons: MAE: 10.26270979642868, MAPE: 0.0823237476829131
1/1 [==============================] - 0s 23ms/step
For 30 neurons: MAE: 16.560689830780035, MAPE: 0.13419359119496793
1/1 [==============================] - 0s 20ms/step
For 40 neurons: MAE: 25.55771850744883, MAPE: 0.20548781501636185
1/1 [==============================] - 0s 29ms/step
For 50 neurons: MAE: 30.087937668959295, MAPE: 0.24149448470142454
Best model for county 26035 had 10 neurons with MAE: 7.320577808221192, MAPE: 0.05939712963718082
Processing county 51155
1/1 [==============================] - 0s 29ms/step
For 10 neurons: MAE: 4.580053794384, MAPE: 0.06584566080217141
1/1 [==============================] - 0s 20ms/step
For 20 neurons: MAE: 8.70654084980489, MAPE: 0.1131517275877784
1/1 [===========================

In [ ]:
#HIV+PCI+SCI
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(1)

# Randomly select 30 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 30, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVSCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVSCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCISCIRandom30d.csv", index=False)

Processing county 26035
1/1 [==============================] - 0s 23ms/step
For 10 neurons: MAE: 9.313852721452724, MAPE: 0.07077988481475317
1/1 [==============================] - 0s 22ms/step
For 20 neurons: MAE: 33.45727901458741, MAPE: 0.2670120847205612
1/1 [==============================] - 0s 21ms/step
For 30 neurons: MAE: 27.4408586661021, MAPE: 0.22130347557591581
1/1 [==============================] - 0s 22ms/step
For 40 neurons: MAE: 66.17987526754538, MAPE: 0.5199489801610726
1/1 [==============================] - 0s 25ms/step
For 50 neurons: MAE: 26.738386269410437, MAPE: 0.21490255895373836
Best model for county 26035 had 10 neurons with MAE: 9.313852721452724, MAPE: 0.07077988481475317
Processing county 51155
1/1 [==============================] - 0s 25ms/step
For 10 neurons: MAE: 3.186250559488935, MAPE: 0.04347201888511427
1/1 [==============================] - 0s 26ms/step
For 20 neurons: MAE: 8.496392786502847, MAPE: 0.10970329853960901
1/1 [=========================

In [ ]:
#HIV+PCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(1)

# Randomly select 30 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 30, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCIDisRandom30d.csv", index=False)

Processing county 26035
1/1 [==============================] - 0s 22ms/step
For 10 neurons: MAE: 10.861886235078194, MAPE: 0.08709277319756013
1/1 [==============================] - 0s 28ms/step
For 20 neurons: MAE: 15.27311843236287, MAPE: 0.11724104555705617
1/1 [==============================] - 0s 23ms/step
For 30 neurons: MAE: 19.589447351296737, MAPE: 0.15202785771110003
1/1 [==============================] - 0s 21ms/step
For 40 neurons: MAE: 53.59212872385979, MAPE: 0.4211306280873279
1/1 [==============================] - 0s 46ms/step
For 50 neurons: MAE: 26.079810313383728, MAPE: 0.20947340461009512
Best model for county 26035 had 10 neurons with MAE: 10.861886235078194, MAPE: 0.08709277319756013
Processing county 51155
1/1 [==============================] - 0s 23ms/step
For 10 neurons: MAE: 5.274707349141441, MAPE: 0.07570869422635128
1/1 [==============================] - 0s 35ms/step
For 20 neurons: MAE: 4.9667030254999815, MAPE: 0.0714145052241112
1/1 [====================

In [ ]:
#HIV+SCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(1)

# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 30, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVDis1', 'HIVSCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVDis1', 'HIVSCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVSCIDisRandom30d.csv", index=False)

Processing county 26035
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 10.987679155667621, MAPE: 0.08365451926344691
1/1 [==============================] - 0s 32ms/step
For 20 neurons: MAE: 13.797668008009586, MAPE: 0.10525962558536005
1/1 [==============================] - 0s 22ms/step
For 30 neurons: MAE: 50.72502820094426, MAPE: 0.40443873586827
1/1 [==============================] - 0s 27ms/step
For 40 neurons: MAE: 14.134001461664846, MAPE: 0.1145776982203313
1/1 [==============================] - 0s 27ms/step
For 50 neurons: MAE: 16.572487394014995, MAPE: 0.1346639900665757
Best model for county 26035 had 10 neurons with MAE: 10.987679155667621, MAPE: 0.08365451926344691
Processing county 51155
1/1 [==============================] - 0s 19ms/step
For 10 neurons: MAE: 5.424751865863802, MAPE: 0.07817282149624195
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 6.123473564783737, MAPE: 0.08860601808544612
1/1 [=======================

In [ ]:
#HIV+PCI+SCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(1)

# Randomly select 30 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 30, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 4)) # 4 because we now have 4 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 4))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 3), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCISCIDisRandom30d.csv", index=False)

Processing county 26035
1/1 [==============================] - 0s 27ms/step
For 10 neurons: MAE: 15.49795258045197, MAPE: 0.12266922208786823
1/1 [==============================] - 0s 28ms/step
For 20 neurons: MAE: 28.85740938385327, MAPE: 0.22309152974978078
1/1 [==============================] - 0s 21ms/step
For 30 neurons: MAE: 8.320826750993723, MAPE: 0.06276603354700128
1/1 [==============================] - 0s 26ms/step
For 40 neurons: MAE: 13.146910194555934, MAPE: 0.1075774803257931


1/1 [==============================] - 0s 20ms/step
For 50 neurons: MAE: 12.899061874548584, MAPE: 0.09922504989563348
Best model for county 26035 had 30 neurons with MAE: 8.320826750993723, MAPE: 0.06276603354700128
Processing county 51155


1/1 [==============================] - 0s 29ms/step
For 10 neurons: MAE: 4.964836744467419, MAPE: 0.07128690889511897
1/1 [==============================] - 0s 24ms/step
For 20 neurons: MAE: 4.851829095681509, MAPE: 0.070053986338776
1/1 [==============================] - 0s 19ms/step
For 30 neurons: MAE: 7.009741353988659, MAPE: 0.08901263987693042
1/1 [==============================] - 0s 21ms/step
For 40 neurons: MAE: 6.249883679548901, MAPE: 0.0898014858227268
1/1 [==============================] - 0s 19ms/step
For 50 neurons: MAE: 15.831227525075292, MAPE: 0.2069767553072185
Best model for county 51155 had 20 neurons with MAE: 4.851829095681509, MAPE: 0.070053986338776
Processing county 55013
1/1 [==============================] - 0s 25ms/step
For 10 neurons: MAE: 5.538815608620642, MAPE: 0.11504595685035257
1/1 [==============================] - 0s 22ms/step
For 20 neurons: MAE: 18.45375923216343, MAPE: 0.3904780776091497
1/1 [==============================] - 0s 23ms/step
For 30

# Randomly select 100 counties

In [ ]:
#####HIV
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)
df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(3)
# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0]

            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year]['HIVprevalence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVRandom100a3.csv", index=False)

Processing county 31055
1/1 [==============================] - 0s 35ms/step
For 10 neurons: MAE: 23.75260264078776, MAPE: 0.3656007897591465
1/1 [==============================] - 0s 32ms/step
For 20 neurons: MAE: 52.99103444417318, MAPE: 0.8501268396416816
1/1 [==============================] - 0s 22ms/step
For 30 neurons: MAE: 30.632714843749998, MAPE: 0.4968926769204393
1/1 [==============================] - 0s 21ms/step
For 40 neurons: MAE: 207.04717407226562, MAPE: 3.3282152219344794


1/1 [==============================] - 0s 24ms/step
For 50 neurons: MAE: 191.99256286621093, MAPE: 3.095253649547285
Best model for county 31055 had 10 neurons with MAE: 23.75260264078776, MAPE: 0.3656007897591465
Processing county 27035


1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 8.459330622355141, MAPE: 0.22456772899024516
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 7.05416399637858, MAPE: 0.18553725000478025
1/1 [==============================] - 0s 22ms/step
For 30 neurons: MAE: 9.114005788167317, MAPE: 0.34587815589177445
1/1 [==============================] - 0s 23ms/step
For 40 neurons: MAE: 10.162892405192055, MAPE: 0.39844587125306213
1/1 [==============================] - 0s 23ms/step
For 50 neurons: MAE: 15.310335032145183, MAPE: 0.6320786836233716
Best model for county 27035 had 20 neurons with MAE: 7.05416399637858, MAPE: 0.18553725000478025
Processing county 51143
1/1 [==============================] - 0s 21ms/step
For 10 neurons: MAE: 12.618446858723965, MAPE: 0.0768305958049689
1/1 [==============================] - 0s 21ms/step
For 20 neurons: MAE: 60.18345642089844, MAPE: 0.3743186909725615
1/1 [==============================] - 0s 25ms/step
Fo

In [ ]:
#HIV+HIVPCI
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(3)

# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCIRandom100b3.csv", index=False)

Processing county 31055
1/1 [==============================] - 0s 23ms/step
For 10 neurons: MAE: 52.83923270304998, MAPE: 0.8169139510667437
1/1 [==============================] - 0s 24ms/step
For 20 neurons: MAE: 80.84718334674835, MAPE: 1.2771033999163104
1/1 [==============================] - 0s 22ms/step
For 30 neurons: MAE: 143.80174859364828, MAPE: 2.2867353698408994
1/1 [==============================] - 0s 26ms/step
For 40 neurons: MAE: 16.491418515642486, MAPE: 0.250891376104506


1/1 [==============================] - 0s 25ms/step
For 50 neurons: MAE: 84.63641238212585, MAPE: 1.3429229893072963
Best model for county 31055 had 40 neurons with MAE: 16.491418515642486, MAPE: 0.250891376104506
Processing county 27035


1/1 [==============================] - 0s 25ms/step
For 10 neurons: MAE: 9.105483440558114, MAPE: 0.24648372211323433
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 7.76839969754219, MAPE: 0.23020671138853263
1/1 [==============================] - 0s 24ms/step
For 30 neurons: MAE: 13.480184690157571, MAPE: 0.5547584143715496
1/1 [==============================] - 0s 23ms/step
For 40 neurons: MAE: 18.761300388971964, MAPE: 0.7516103342585594
1/1 [==============================] - 0s 24ms/step
For 50 neurons: MAE: 24.588068707784018, MAPE: 0.9675148092258401
Best model for county 27035 had 20 neurons with MAE: 7.76839969754219, MAPE: 0.23020671138853263
Processing county 51143
1/1 [==============================] - 0s 23ms/step
For 10 neurons: MAE: 29.00776042938233, MAPE: 0.1768720907195173
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 77.46778165499371, MAPE: 0.4826837609913763
1/1 [==============================] - 0s 22ms/step
For 

In [ ]:
#HIV+HIVSCI
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(3)

# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVSCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVSCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVSCIRandom100c3.csv", index=False)

Processing county 31055
1/1 [==============================] - 0s 72ms/step
For 10 neurons: MAE: 50.05761471390724, MAPE: 0.7740257473193788
1/1 [==============================] - 0s 44ms/step
For 20 neurons: MAE: 141.8898377418518, MAPE: 2.2538369503884605
1/1 [==============================] - 0s 34ms/step
For 30 neurons: MAE: 104.58652369181316, MAPE: 1.659113423602216
1/1 [==============================] - 0s 39ms/step
For 40 neurons: MAE: 74.56536463896434, MAPE: 1.182594152733093


1/1 [==============================] - 0s 76ms/step
For 50 neurons: MAE: 91.66105051835378, MAPE: 1.4561009377468856
Best model for county 31055 had 10 neurons with MAE: 50.05761471390724, MAPE: 0.7740257473193788
Processing county 27035


1/1 [==============================] - 0s 37ms/step
For 10 neurons: MAE: 6.589458968242007, MAPE: 0.17595127511979014
1/1 [==============================] - 0s 22ms/step
For 20 neurons: MAE: 7.3003961344559976, MAPE: 0.20578060862803862
1/1 [==============================] - 0s 26ms/step
For 30 neurons: MAE: 8.537742114067076, MAPE: 0.2869984933080438
1/1 [==============================] - 0s 24ms/step
For 40 neurons: MAE: 14.01702624956767, MAPE: 0.5849471622329985
1/1 [==============================] - 0s 26ms/step
For 50 neurons: MAE: 18.071596018473304, MAPE: 0.7321085538057078
Best model for county 27035 had 10 neurons with MAE: 6.589458968242007, MAPE: 0.17595127511979014
Processing county 51143
1/1 [==============================] - 0s 37ms/step
For 10 neurons: MAE: 15.170202430089327, MAPE: 0.09265128847180455
1/1 [==============================] - 0s 22ms/step
For 20 neurons: MAE: 25.482829316457117, MAPE: 0.16026801057341164
1/1 [==============================] - 0s 35ms/step

In [ ]:
#HIV+HIVDis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(3)

# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2)) # 2 because we now have 2 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 2))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 1), train[-1, 1]))))[:, 0][0]

            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVDisRandom100d3.csv", index=False)

Processing county 31055
1/1 [==============================] - 0s 31ms/step
For 10 neurons: MAE: 11.07176286230485, MAPE: 0.171965294899362
1/1 [==============================] - 0s 36ms/step
For 20 neurons: MAE: 16.937643041213356, MAPE: 0.2599023891753128
1/1 [==============================] - 0s 47ms/step
For 30 neurons: MAE: 21.017000099023186, MAPE: 0.3228923377122007
1/1 [==============================] - 0s 37ms/step
For 40 neurons: MAE: 49.96236105759939, MAPE: 0.7868789516757472


1/1 [==============================] - 0s 35ms/step
For 50 neurons: MAE: 11.327754000822708, MAPE: 0.20040786459048862
Best model for county 31055 had 10 neurons with MAE: 11.07176286230485, MAPE: 0.171965294899362
Processing county 27035


1/1 [==============================] - 0s 51ms/step
For 10 neurons: MAE: 6.418955010175703, MAPE: 0.1755293392166654
1/1 [==============================] - 0s 35ms/step
For 20 neurons: MAE: 7.296759299437203, MAPE: 0.20378843793409063
1/1 [==============================] - 0s 21ms/step
For 30 neurons: MAE: 11.689644304911292, MAPE: 0.48558520317484005
1/1 [==============================] - 0s 25ms/step
For 40 neurons: MAE: 23.016443339983624, MAPE: 0.9038026949373866
1/1 [==============================] - 0s 22ms/step
For 50 neurons: MAE: 39.737996117273966, MAPE: 1.5053894187547405
Best model for county 27035 had 10 neurons with MAE: 6.418955010175703, MAPE: 0.1755293392166654
Processing county 51143
1/1 [==============================] - 0s 54ms/step
For 10 neurons: MAE: 27.71948825518292, MAPE: 0.16978976145775973
1/1 [==============================] - 0s 26ms/step
For 20 neurons: MAE: 60.87920660972597, MAPE: 0.3798105063651802
1/1 [==============================] - 0s 37ms/step
Fo

In [ ]:
#HIV+PCI+SCI
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(3)

# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVSCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVSCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCISCIRandom100d33.csv", index=False)

Processing county 31055
1/1 [==============================] - 0s 82ms/step
For 10 neurons: MAE: 13.772576818863554, MAPE: 0.2124875572607382
1/1 [==============================] - 0s 32ms/step
For 20 neurons: MAE: 38.289536575476326, MAPE: 0.5984294487360851
1/1 [==============================] - 0s 34ms/step
For 30 neurons: MAE: 45.75075217088064, MAPE: 0.7206350894852628
1/1 [==============================] - 0s 69ms/step
For 40 neurons: MAE: 57.528191367785126, MAPE: 0.9346229326957151


1/1 [==============================] - 0s 23ms/step
For 50 neurons: MAE: 73.78474303086598, MAPE: 1.1691302548440776
Best model for county 31055 had 10 neurons with MAE: 13.772576818863554, MAPE: 0.2124875572607382
Processing county 27035


1/1 [==============================] - 0s 26ms/step
For 10 neurons: MAE: 7.770628108580907, MAPE: 0.20858376843805293
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 7.738040608167648, MAPE: 0.20363627185230074
1/1 [==============================] - 0s 24ms/step
For 30 neurons: MAE: 7.4774827559789, MAPE: 0.21022520366310593
1/1 [==============================] - 0s 24ms/step
For 40 neurons: MAE: 8.131678386529286, MAPE: 0.21068353471288162
1/1 [==============================] - 0s 23ms/step
For 50 neurons: MAE: 8.609621596336362, MAPE: 0.27755737705762734
Best model for county 27035 had 20 neurons with MAE: 7.738040608167648, MAPE: 0.20363627185230074
Processing county 51143
1/1 [==============================] - 0s 25ms/step
For 10 neurons: MAE: 7.7400505463282245, MAPE: 0.04827206844509552
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 18.3610905567805, MAPE: 0.11089212871444831
1/1 [==============================] - 0s 45ms/step
Fo

In [ ]:
#HIV+PCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(2)

# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCIDisRandom100d.csv", index=False)

Processing county 36025
1/1 [==============================] - 0s 25ms/step
For 10 neurons: MAE: 1.7213559826215128, MAPE: 0.013476389226611712
1/1 [==============================] - 0s 19ms/step
For 20 neurons: MAE: 23.31313111043224, MAPE: 0.18125805565598543
1/1 [==============================] - 0s 21ms/step
For 30 neurons: MAE: 6.711706672112139, MAPE: 0.05206244183211012
1/1 [==============================] - 0s 39ms/step
For 40 neurons: MAE: 3.849972405036302, MAPE: 0.0298791486152729


1/1 [==============================] - 0s 23ms/step
For 50 neurons: MAE: 1.1629000584284388, MAPE: 0.008963420310207502
Best model for county 36025 had 50 neurons with MAE: 1.1629000584284388, MAPE: 0.008963420310207502
Processing county 37101


1/1 [==============================] - 0s 33ms/step
For 10 neurons: MAE: 7.0456458528836565, MAPE: 0.029163105813179362
1/1 [==============================] - 0s 20ms/step
For 20 neurons: MAE: 26.958061782519014, MAPE: 0.11271126979516694
1/1 [==============================] - 0s 20ms/step
For 30 neurons: MAE: 4.5352546493212555, MAPE: 0.018859038885722157
1/1 [==============================] - 0s 21ms/step
For 40 neurons: MAE: 28.185047590235865, MAPE: 0.1180674518547905
1/1 [==============================] - 0s 28ms/step
For 50 neurons: MAE: 15.06209996740022, MAPE: 0.06298065342529537
Best model for county 37101 had 30 neurons with MAE: 4.5352546493212555, MAPE: 0.018859038885722157
Processing county 38015
1/1 [==============================] - 0s 20ms/step
For 10 neurons: MAE: 7.56640142997107, MAPE: 0.10454151001828528
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 8.068282802899688, MAPE: 0.11167089014148435
1/1 [==============================] - 0s 21ms

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



For 50 neurons: MAE: 586.9636956691743, MAPE: 4.650031197602849
Best model for county 48179 had 10 neurons with MAE: 16.4100184271733, MAPE: 0.12350707575957763
Average MAE across all counties: 14.38983558218212
Average MAPE across all counties: 0.08332116956478522
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-2ac360e4ab94>", line 102, in <cell line: 102>
    predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCIDisRandom100d.csv", index=False)
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 3720, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-2ac360e4ab94>", line 102, in <cell line: 102>
    predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCIDisRandom100d.csv", index=False)
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 3720, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py", line 1189, in to_csv
    csv_formatter.save()
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/formats/csvs.py", line 241, in save
    with get_h

In [ ]:
#HIV+SCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(2)

# Randomly select 50 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVDis1', 'HIVSCI1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3)) # 3 because we now have 3 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 3))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 2), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVDis1', 'HIVSCI1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVSCIDisRandom100d.csv", index=False)

In [ ]:
#HIV+PCI+SCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(3)

# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 4)) # 4 because we now have 4 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 4))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 3), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCISCIDisRandom100d3.csv", index=False)

Processing county 31055
1/1 [==============================] - 0s 36ms/step
For 10 neurons: MAE: 11.004789308955274, MAPE: 0.16534854175764613
1/1 [==============================] - 0s 34ms/step
For 20 neurons: MAE: 6.843323438117902, MAPE: 0.11512029286621202
1/1 [==============================] - 0s 41ms/step
For 30 neurons: MAE: 42.10749888420105, MAPE: 0.6641892491610667
1/1 [==============================] - 0s 23ms/step
For 40 neurons: MAE: 110.75513799985248, MAPE: 1.7848095668951096


1/1 [==============================] - 0s 25ms/step
For 50 neurons: MAE: 79.5919256210327, MAPE: 1.2917827132617252
Best model for county 31055 had 20 neurons with MAE: 6.843323438117902, MAPE: 0.11512029286621202
Processing county 27035


1/1 [==============================] - 0s 25ms/step
For 10 neurons: MAE: 7.207075629631677, MAPE: 0.2124332196812538
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 8.185906930764515, MAPE: 0.2633664235714025
1/1 [==============================] - 0s 21ms/step
For 30 neurons: MAE: 7.836628339687981, MAPE: 0.24117464869070618
1/1 [==============================] - 0s 27ms/step
For 40 neurons: MAE: 8.530833647648493, MAPE: 0.2229925482666705
1/1 [==============================] - 0s 24ms/step
For 50 neurons: MAE: 13.015373857816058, MAPE: 0.5434229117927506
Best model for county 27035 had 10 neurons with MAE: 7.207075629631677, MAPE: 0.2124332196812538
Processing county 51143
1/1 [==============================] - 0s 22ms/step
For 10 neurons: MAE: 13.38502754370372, MAPE: 0.08186772726164597
1/1 [==============================] - 0s 23ms/step
For 20 neurons: MAE: 23.198032565911614, MAPE: 0.14037931184903316
1/1 [==============================] - 0s 23ms/step
For

# Randomly select 100 counties 2

In [ ]:
#####HIV
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)
df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(4)
# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0]

            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year]['HIVprevalence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVRandom100a34.csv", index=False)

Processing county 29155
1/1 [==============================] - 0s 67ms/step
For 10 neurons: MAE: 9.890535481770835, MAPE: 0.07383815772109704
1/1 [==============================] - 0s 48ms/step
For 20 neurons: MAE: 22.986848958333336, MAPE: 0.16368494914355783
1/1 [==============================] - 0s 93ms/step
For 30 neurons: MAE: 42.412269083658856, MAPE: 0.2951472678577705
1/1 [==============================] - 0s 111ms/step
For 40 neurons: MAE: 55.102881876627606, MAPE: 0.3815678064701949


1/1 [==============================] - 0s 83ms/step
For 50 neurons: MAE: 93.55411478678384, MAPE: 0.6423065972415575
Best model for county 29155 had 10 neurons with MAE: 9.890535481770835, MAPE: 0.07383815772109704
Processing county 55087


1/1 [==============================] - 0s 63ms/step
For 10 neurons: MAE: 16.06665318806966, MAPE: 0.24441680573806304
1/1 [==============================] - 0s 30ms/step
For 20 neurons: MAE: 12.210394541422525, MAPE: 0.20727206322445246
1/1 [==============================] - 0s 29ms/step
For 30 neurons: MAE: 13.327154795328775, MAPE: 0.238836564220655
1/1 [==============================] - 0s 39ms/step
For 40 neurons: MAE: 31.836165873209637, MAPE: 0.6133540927796766
1/1 [==============================] - 0s 34ms/step
For 50 neurons: MAE: 65.1339360555013, MAPE: 1.226770989428513
Best model for county 55087 had 20 neurons with MAE: 12.210394541422525, MAPE: 0.20727206322445246
Processing county 37089
1/1 [==============================] - 0s 43ms/step
For 10 neurons: MAE: 26.614718627929687, MAPE: 0.17797191239104518
1/1 [==============================] - 0s 27ms/step
For 20 neurons: MAE: 51.75793253580729, MAPE: 0.35238200906797995
1/1 [==============================] - 0s 21ms/step
F

In [ ]:
#HIV+PCI+SCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(4)

# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 4)) # 4 because we now have 4 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 4))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 3), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCISCIDisRandom100d34.csv", index=False)

Processing county 29155
1/1 [==============================] - 0s 27ms/step
For 10 neurons: MAE: 13.641565301020941, MAPE: 0.09977847632754984
1/1 [==============================] - 0s 28ms/step
For 20 neurons: MAE: 9.436923964818325, MAPE: 0.06898100888060446
1/1 [==============================] - 0s 31ms/step
For 30 neurons: MAE: 7.731083558003121, MAPE: 0.055319459051614304
1/1 [==============================] - 0s 30ms/step
For 40 neurons: MAE: 21.791988734404242, MAPE: 0.1533687689472569


1/1 [==============================] - 0s 26ms/step
For 50 neurons: MAE: 10.225271308422094, MAPE: 0.0751982374329154
Best model for county 29155 had 30 neurons with MAE: 7.731083558003121, MAPE: 0.055319459051614304
Processing county 55087


1/1 [==============================] - 0s 30ms/step
For 10 neurons: MAE: 9.705079569419226, MAPE: 0.16021950013782357
1/1 [==============================] - 0s 21ms/step
For 20 neurons: MAE: 20.119035387039187, MAPE: 0.40520767248358097
1/1 [==============================] - 0s 20ms/step
For 30 neurons: MAE: 15.25161829292774, MAPE: 0.22996159373773647
1/1 [==============================] - 0s 18ms/step
For 40 neurons: MAE: 17.737755931417144, MAPE: 0.2568818719783082
1/1 [==============================] - 0s 20ms/step
For 50 neurons: MAE: 12.88480013012886, MAPE: 0.22073008305057742
Best model for county 55087 had 10 neurons with MAE: 9.705079569419226, MAPE: 0.16021950013782357
Processing county 37089
1/1 [==============================] - 0s 24ms/step
For 10 neurons: MAE: 14.314000956217447, MAPE: 0.09773337471290333
1/1 [==============================] - 0s 18ms/step
For 20 neurons: MAE: 23.45848535696665, MAPE: 0.16257587985961205
1/1 [==============================] - 0s 19ms/ste

# Randomly select 100 counties 3

In [ ]:
#####HIV
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)
df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(5)
# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')

    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 1 because we now have 1 feature

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 1)) # 1 because we now have 1 feature

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(pred_single)[0][0]

            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year]['HIVprevalence'].values
                train = np.concatenate((train, next_year_data.reshape(-1, 1)))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVRandom100a345.csv", index=False)

FileNotFoundError: ignored

In [ ]:
#HIV+PCI+SCI+Dis
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

np.random.seed(1053)
tf.random.set_seed(1053)

df = pd.read_csv("TotalData.csv")

df['County'] = df['County'].astype('category')

all_predictions = []
all_maes = []
all_mapes = []

# Set the seed for the random number generator
np.random.seed(5)

# Randomly select 100 counties
unique_counties = df['County'].unique()
selected_counties = np.random.choice(unique_counties, 100, replace=False)

for county in selected_counties:
    print(f'Processing county {county}')
    county_df = df[df['County'] == county]

    train_df = county_df[(county_df['Year'] >= 2008) & (county_df['Year'] < 2019)]

    scaler = MinMaxScaler()
    train = scaler.fit_transform(train_df[['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']])

    x_train = np.array([train[i-9:i] for i in range(9, len(train))])
    y_train = np.array([train[i, 0] for i in range(9, len(train))])

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 4)) # 4 because we now have 4 features

    best_mape = float('inf')  # start with a high MAPE
    best_predictions = None
    best_neurons = None
    best_mae = None

    # Loop over all possible number of neurons
    for neurons in range(10, 51, 10):
        tf.keras.backend.clear_session()  # Clear the previous model to free up memory
        model = Sequential()
        model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(LSTM(neurons, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=100, verbose=0)

        # Rest of the code remains the same
        predictions = []
        for year in range(2017, 2020):
            input_data = train[-9:]  # take the last 9 years as input data
            input_data = input_data.reshape((1, 9, 4))

            pred_single = model.predict(input_data)
            pred_single_transformed_back = scaler.inverse_transform(np.hstack((pred_single, np.full((1, 3), train[-1, 1:]))))[:, 0]
            # Save prediction instead of replacing with true value
            predictions.append((county, year, pred_single_transformed_back))

            # If we have the true data for the next year, use it to update the training set
            if year in train_df['Year'].values:
                next_year_data = train_df.loc[train_df['Year'] == year][['HIVprevalence', 'HIVXPCI1', 'HIVSCI1', 'HIVDis1']].values
                train = np.concatenate((train, next_year_data))

        true_values = county_df[(county_df['Year'] >= 2017) & (county_df['Year'] <= 2019)]['HIVprevalence'].values
        predicted_values = np.array([pred[2] for pred in predictions])

        mae = mean_absolute_error(true_values, predicted_values[-len(true_values):])
        mape = mean_absolute_percentage_error(true_values, predicted_values[-len(true_values):])

        print(f'For {neurons} neurons: MAE: {mae}, MAPE: {mape}')

        if mape < best_mape:
            best_mape = mape
            best_predictions = predictions
            best_neurons = neurons
            best_mae = mae

    all_predictions.extend(best_predictions)
    all_maes.append(best_mae)
    all_mapes.append(best_mape)

    print(f'Best model for county {county} had {best_neurons} neurons with MAE: {best_mae}, MAPE: {best_mape}')

average_mae = np.mean(all_maes)
average_mape = np.mean(all_mapes)

print(f'Average MAE across all counties: {average_mae}')
print(f'Average MAPE across all counties: {average_mape}')

predictions_df = pd.DataFrame(all_predictions, columns=['County', 'Year', 'Predicted_HIV_Rate'])
predictions_df.to_csv("MovingWindowsTotalDataPredictionHIVPCISCIDisRandom100d345.csv", index=False)